In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import tqdm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm


In [2]:
df = pd.read_csv('Ad_table (extra).csv')
print(f"Всего записей: {len(df)}")
print(df['Color'].value_counts())

Всего записей: 268255
Color
Black          48751
Silver         40214
Blue           38376
Grey           37678
White          34270
Red            25987
Green           5027
Yellow          3072
Brown           2878
Orange          2829
Beige           1982
Purple          1361
Gold            1223
Bronze          1200
Multicolour      800
Pink             299
Maroon           179
Turquoise        176
Burgundy          48
Magenta           18
Navy               8
Indigo             4
Name: count, dtype: int64


In [3]:
def explore_image_directory(root_dir):
    car_images = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                car_images.append(os.path.join(root, file))
    
    print(f"Найдено {len(car_images)} изображений")
    return car_images

car_images = explore_image_directory('confirmed_fronts')
print(f"Пример пути к изображению: {car_images[0]}")

Найдено 61827 изображений
Пример пути к изображению: confirmed_fronts\Abarth\2013\Abarth$$595$$2013$$Black$$2_4$$100$$image_1.jpg


In [4]:
def extract_color_from_filename(filename):
    # Abarth $$ 595 $$ 2013 $$ Black $$ 2_4 $$ 100 $$ image_1.jpg
    parts = os.path.basename(filename).split('$$')
    if len(parts) >= 4:
        return parts[3]
    return None

image_colors = {}
for img_path in car_images:
    color = extract_color_from_filename(img_path)
    if color:
        image_colors[img_path] = color

print(f"Найдено {len(image_colors)} изображений с цветами")
color_distribution = pd.Series(list(image_colors.values())).value_counts()
print(color_distribution)

Найдено 61827 изображений с цветами
Black          14317
Grey            9474
White           9395
Blue            8483
Silver          7770
Red             6095
Unlisted        1516
Brown            911
Green            777
Yellow           667
Beige            600
Orange           559
Purple           362
Bronze           329
Gold             217
Multicolour      196
Pink              87
Turquoise         26
Maroon            26
Burgundy           9
Magenta            9
Navy               1
Indigo             1
Name: count, dtype: int64


In [5]:
class CarColorDataset(Dataset):
    def __init__(self, image_paths, colors, transform=None):
        self.image_paths = image_paths
        self.colors = colors
        self.transform = transform
        
        unique_colors = list(set(colors))
        self.color_to_idx = {color: idx for idx, color in enumerate(unique_colors)}
        self.idx_to_color = {idx: color for idx, color in enumerate(unique_colors)}
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        color = self.colors[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, self.color_to_idx[color]

image_paths = list(image_colors.keys())
colors = list(image_colors.values())

color_counts = pd.Series(colors).value_counts()
print("распределение цветов в датасете:")
print(color_counts)

min_samples_per_class = 3
valid_colors = color_counts[color_counts >= min_samples_per_class].index.tolist()
print(f"\nцвета с как минимум {min_samples_per_class} образцами: {valid_colors}")

filtered_indices = [i for i, color in enumerate(colors) if color in valid_colors]
filtered_image_paths = [image_paths[i] for i in filtered_indices]
filtered_colors = [colors[i] for i in filtered_indices]

print(f"размер исходного датасета: {len(colors)} изображений")
print(f"размер отфильтрованного датасета: {len(filtered_colors)} изображений")

def map_rare_colors(color, min_count=3):
    if color_counts[color] < min_count:
        return "other"
    return color

grouped_image_paths = image_paths.copy()
grouped_colors = [map_rare_colors(color) for color in colors]

grouped_color_counts = pd.Series(grouped_colors).value_counts()
print("\nраспределение цветов после группировки:")
print(grouped_color_counts)

use_filtering = False

if use_filtering:
    processed_image_paths = filtered_image_paths
    processed_colors = filtered_colors
else:
    processed_image_paths = grouped_image_paths
    processed_colors = grouped_colors

print(f"\nиспользуем {'фильтрацию редких цветов' if use_filtering else 'группировку редких цветов в Other'}")
print(f"итоговый размер датасета: {len(processed_colors)} изображений")

X_train_val, X_test, y_train_val, y_test = train_test_split(
    processed_image_paths, processed_colors, test_size=0.15, random_state=42, stratify=processed_colors
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.15, random_state=42, stratify=y_train_val
)

print(f"размер обучающей выборки: {len(X_train)}")
print(f"размер валидационной выборки: {len(X_val)}")
print(f"размер тестовой выборки: {len(X_test)}")

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = CarColorDataset(X_train, y_train, transform=train_transform)
val_dataset = CarColorDataset(X_val, y_val, transform=val_test_transform)
test_dataset = CarColorDataset(X_test, y_test, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

num_classes = len(train_dataset.color_to_idx)
print(f"всего классов (цветов): {num_classes}")
print(f"маппинг цветов: {train_dataset.color_to_idx}")

распределение цветов в датасете:
Black          14317
Grey            9474
White           9395
Blue            8483
Silver          7770
Red             6095
Unlisted        1516
Brown            911
Green            777
Yellow           667
Beige            600
Orange           559
Purple           362
Bronze           329
Gold             217
Multicolour      196
Pink              87
Turquoise         26
Maroon            26
Burgundy           9
Magenta            9
Navy               1
Indigo             1
Name: count, dtype: int64

цвета с как минимум 3 образцами: ['Black', 'Grey', 'White', 'Blue', 'Silver', 'Red', 'Unlisted', 'Brown', 'Green', 'Yellow', 'Beige', 'Orange', 'Purple', 'Bronze', 'Gold', 'Multicolour', 'Pink', 'Turquoise', 'Maroon', 'Burgundy', 'Magenta']
размер исходного датасета: 61827 изображений
размер отфильтрованного датасета: 61825 изображений

распределение цветов после группировки:
Black          14317
Grey            9474
White           9395
Blue           

`ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.`

In [6]:
def create_resnet_from_scratch(num_classes):
    # ResNet50 случайные веса
    model = models.resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

model_from_scratch = create_resnet_from_scratch(num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device.type)
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)

model_from_scratch = model_from_scratch.to(device)

# функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_from_scratch.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)



C:\Users\atheb\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\atheb\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


cuda
2.6.0+cu126
True
12.6


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=25):
    train_losses = []
    val_losses = []
    train_f1_scores = []
    val_f1_scores = []
    
    best_val_f1 = 0.0
    best_model_weights = None
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        pbar = tqdm(train_loader, desc=f'epoch {epoch+1}/{num_epochs} [Train]')
        for inputs, labels in pbar:
            print(inputs.device)

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            
            pbar.set_postfix({'loss': loss.item()})
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_f1 = f1_score(all_labels, all_preds, average='macro')
        
        # валидация
        model.eval()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                print(inputs.device)
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                running_loss += loss.item() * inputs.size(0)
                
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_f1 = f1_score(all_labels, all_preds, average='macro')
        
        scheduler.step(epoch_val_loss)
        
        if epoch_val_f1 > best_val_f1:
            best_val_f1 = epoch_val_f1
            best_model_weights = model.state_dict().copy()
        
        train_losses.append(epoch_train_loss)
        val_losses.append(epoch_val_loss)
        train_f1_scores.append(epoch_train_f1)
        val_f1_scores.append(epoch_val_f1)
        
        print(f'epoch {epoch+1}/{num_epochs}')
        print(f'train Loss: {epoch_train_loss:.4f}, train F1: {epoch_train_f1:.4f}')
        print(f'val Loss: {epoch_val_loss:.4f}, val F1: {epoch_val_f1:.4f}')
        print('-' * 60)
    
    model.load_state_dict(best_model_weights)
    return model, train_losses, val_losses, train_f1_scores, val_f1_scores

print("обучение ResNet50 с нуля...")
model_from_scratch, train_losses_scratch, val_losses_scratch, train_f1_scratch, val_f1_scratch = train_model(
    model_from_scratch, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20
)

обучение ResNet50 с нуля...


epoch 1/20 [Train]:   0%|          | 0/1396 [00:00<?, ?it/s]